In [1]:
# -*- coding: utf-8 -*-
"""
Created on Wed Mar  1 19:09:26 2017

@author: user
"""


import pyodbc
import jieba

from math import*
import codecs
from nltk import word_tokenize
import os #讀取程式目前路徑用
import sys #為了能夠吃進主程式外部輸進的參數
import nltk #引入自然語言處理模組
import io #檔案io
from nltk.corpus import stopwords #去無用字
import re #需要用正則表示式移除標點與數字
from nltk.corpus import wordnet as wn #2013/10/26改用wordnet來做字根還原
from nltk.tokenize import word_tokenize
import json
import numpy as np
import numpy.linalg as LA
import string
import csv
from pandas import Series, DataFrame
import pandas as pd
print("-----STAR-----")
print("try to link with DataBase:cate_test......")
cnxn = pyodbc.connect("DRIVER={SQL Server};SERVER=LEAVES-PC\SA000;DATABASE=cate_test;UID=sa;PWD=00000000")
cursor = cnxn.cursor()
print("sccessful link")

cursor.execute("select [BulletinPlatform] from [cate_test].[dbo].[20161020001_all_candinates]")
results = cursor.fetchall()


count=0 
for i in results:
    count+=1
    #print ('%d:%s' % (count, i))
print("getting data sccessful")

def getDocDistance(a, b):
    if LA.norm(a)==0 or LA.norm(b)==0:
        return -1  
    return round(np.inner(a, b) / (LA.norm(a) * LA.norm(b)), 5) 
def getDocSimilarity(wordFrequencyPair, minTimes=1):
    #v1 = var1
    #v2 = var2
    result = 0
    try:
        result = getDocDistance(v1, v2)
    except(RuntimeError, TypeError, NameError):
        pass        
    return result

with open('new_PFP_20161020001.csv', 'w', newline='') as f: 
    spamwriter = csv.writer(f, dialect='excel')
    headers = ['i', 'j', 'Similarity','i text','j text']
    spamwriter.writerow(headers)
    print("the csv document has created ")
    print("processing...........")
    for i in range(0 , len(results)):
        for j in range(i+1 ,len(results)):      
            for word1 in results[i]:                      
                deletepunc = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
                for chin in deletepunc:
                    word1 = word1.replace(chin, ' ')
                result= jieba.cut(word1) 
                result1 = " ".join(result)
                file1 = nltk.word_tokenize(result1)
                trainTokenFrequency = nltk.FreqDist(file1)              
            for word2 in results[j]:             
                deletepunc = [u'〔',u'〕',u'〝',u'〞',u'『 ',u'』',u'〈',u'〉',u'\\',u'（', u'）',u'～',u'u3000',u'"',u'&gt',u'&lt',u';',u'，', u'。', u'、', u'；', u'：', u'?', u'「', u'」', u'%', u'.', u',', u'？', u'-', u'~',u'!',u'！', u'&nbsp', u'<BR>', u'“', u'”', u'【', u'】', u'《', u'》',u'：',u'●',u'(',u')',u'；']
                for chin in deletepunc:
                    word2 = word2.replace(chin, ' ')
                result= jieba.cut(word2) 
                result2 = " ".join(result)
                file2 = nltk.word_tokenize(result2)
                testTokenFrequency = nltk.FreqDist(file2) 
            
            wordFrequencyPair = [trainTokenFrequency, testTokenFrequency]
            
            minTimes=1
            dict1 = {}
            for key in wordFrequencyPair[0].keys():
                if wordFrequencyPair[0].get(key, 0) >= minTimes:
                    dict1[key] = wordFrequencyPair[0].get(key, 0)
            dict2 = {}
            for key in wordFrequencyPair[1].keys():
                if wordFrequencyPair[1].get(key, 0) >= minTimes:
                    dict2[key] = wordFrequencyPair[1].get(key, 0)
            for key in dict2.keys():
                if dict1.get(key, 0) == 0:   #第一個字典中如果沒有第二字典中的字就把沒有的字加進去並設為0
                    dict1[key] = 0
            for key in dict1.keys():
                if dict2.get(key, 0) == 0:
                    dict2[key] = 0
            v1 = []
            for w1 in sorted(dict1.keys()):
                v1.append(dict1.get(w1))
            v2 = []    
            for w2 in sorted(dict2.keys()):
                v2.append(dict2.get(w2))
                
            def getDocSimilarity(wordFrequencyPair, minTimes=1):
                result = 0
                try:                
                    result = getDocDistance(v1, v2)
                    #print("getDocDistance try")
                except(RuntimeError, TypeError, NameError):
                    pass
                return result    
            data = [( i , j , getDocSimilarity(wordFrequencyPair, 1),result1,result2)]   
            spamwriter.writerows(data)       #寫入csv
print("-----FINISH-----")
f.close()

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Leaves\AppData\Local\Temp\jieba.cache


-----STAR-----
try to link with DataBase:cate_test......
sccessful link
getting data sccessful
the csv document has created 
processing...........


Loading model cost 0.928 seconds.
Prefix dict has been built succesfully.


-----FINISH-----
